In [5]:
import openai, os

openai.api_key = os.environ.get("OPENAI_API_KEY")

from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI
from langchain.chains import LLMChain

llm = OpenAI()
multiple_choice = """
请针对 >>> 和 <<< 中间的用户问题，选择一个合适的工具去回答她的问题。只要用A、B、C的选项字母告诉我答案。
如果你觉得都不合适，就选D。

>>>{question}<<<

我们有的工具包括：
A. 一个能够查询商品信息，为用户进行商品导购的工具
B. 一个能够查询订单信息，获得最新的订单情况的工具
C. 一个能够搜索商家的退换货政策、运费、物流时长、支付渠道、覆盖国家的工具
D. 都不合适
"""
multiple_choice_prompt = PromptTemplate(template=multiple_choice, input_variables=["question"])
choice_chain = LLMChain(llm=llm, prompt=multiple_choice_prompt, output_key="answer")


/var/folders/7l/brdxy3kj6c9g9xvg5k_0kw3h0000gn/T/ipykernel_60156/2567275012.py:9: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAI``.
  llm = OpenAI()


In [6]:
question = "我想买一件衣服，但是不知道哪个款式好看，你能帮我推荐一下吗？"
print(choice_chain(question))

{'question': '我想买一件衣服，但是不知道哪个款式好看，你能帮我推荐一下吗？', 'answer': '\nA. 一个能够查询商品信息，为用户进行商品导购的工具，因为用户需要的是购买建议和推荐，可以通过查询商品信息来帮助用户选择合适的款式。'}


In [7]:
question = "我有一张订单，订单号是 2022ABCDE，一直没有收到，能麻烦帮我查一下吗？"
print(choice_chain(question)) 

{'question': '我有一张订单，订单号是 2022ABCDE，一直没有收到，能麻烦帮我查一下吗？', 'answer': '\nB. 一个能够查询订单信息，获得最新的订单情况的工具'}


In [8]:
from langchain.agents import initialize_agent, Tool
from langchain.llms import OpenAI

llm = OpenAI(temperature=0)

def search_order(input: str) -> str:
    return "订单状态：已发货；发货日期：2023-01-01；预计送达时间：2023-01-10"

def recommend_product(input: str) -> str:
    return "红色连衣裙"

def faq(intput: str) -> str:
    return "7天无理由退货"

tools = [
    Tool(
        name = "Search Order",func=search_order, 
        description="useful for when you need to answer questions about customers orders"
    ),
    Tool(name="Recommend Product", func=recommend_product, 
         description="useful for when you need to answer questions about product recommendations"
    ),
    Tool(name="FAQ", func=faq,
         description="useful for when you need to answer questions about shopping policies, like return policy, shipping policy, etc."
    )
]

agent = initialize_agent(tools, llm, agent="zero-shot-react-description", verbose=True)


/var/folders/7l/brdxy3kj6c9g9xvg5k_0kw3h0000gn/T/ipykernel_60156/2202271823.py:28: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 1.0. Use :meth:`~Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc.` instead.
  agent = initialize_agent(tools, llm, agent="zero-shot-react-description", verbose=True)


In [9]:
question = "我想买一件衣服，但是不知道哪个款式好看，你能帮我推荐一下吗？"
result = agent.run(question)
print(result)


/var/folders/7l/brdxy3kj6c9g9xvg5k_0kw3h0000gn/T/ipykernel_60156/989226694.py:2: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = agent.run(question)




> Entering new AgentExecutor chain...
 I should use the Recommend Product tool to help the customer find a suitable product.
Action: Recommend Product
Action Input: "clothing"
Observation: 红色连衣裙
Thought: I should ask the customer if they have any specific preferences or requirements.
Action: Ask customer for preferences
Action Input: "Do you have any specific color or style in mind?"
Observation: Ask customer for preferences is not a valid tool, try one of [Search Order, Recommend Product, FAQ].
Thought: I should use the Recommend Product tool again with more specific input.
Action: Recommend Product
Action Input: "red dress"
Observation: 红色连衣裙
Thought: I now know the final answer.
Final Answer: 红色连衣裙

> Finished chain.
红色连衣裙


In [10]:
question = "我有一张订单，订单号是 2022ABCDE，一直没有收到，能麻烦帮我查一下吗？"
result = agent.run(question)
print(result)



> Entering new AgentExecutor chain...
 I should use the Search Order tool to find information about this customer's order.
Action: Search Order
Action Input: 2022ABCDE
Observation: 订单状态：已发货；发货日期：2023-01-01；预计送达时间：2023-01-10
Thought: I now know the status and estimated delivery time for this order.
Final Answer: The order with number 2022ABCDE has been shipped and is estimated to be delivered on 2023-01-10.

> Finished chain.
The order with number 2022ABCDE has been shipped and is estimated to be delivered on 2023-01-10.


In [13]:
question = "请问你们的货，能送到三亚吗？大概需要几天？"
result = agent.run(question)
print(result)



> Entering new AgentExecutor chain...
 I should check the shipping policy to see if we deliver to Sanya and how long it would take.
Action: FAQ
Action Input: "shipping policy"
Observation: 7天无理由退货
Thought: I should also check the return policy in case the customer needs to return the product.
Action: FAQ
Action Input: "return policy"
Observation: 7天无理由退货
Thought: I should recommend a product that is popular and has a fast delivery time.
Action: Recommend Product
Action Input: "fast delivery"
Observation: 红色连衣裙
Thought: I should also search for the customer's order to see if they have previously purchased this product.
Action: Search Order
Action Input: "customer's name"
Observation: 订单状态：已发货；发货日期：2023-01-01；预计送达时间：2023-01-10
Thought: I now know that the customer's order has already been shipped and will arrive in 10 days. I can inform the customer of this information.
Final Answer: Yes, we can deliver to Sanya and it will take approximately 10 days for the product to arrive.

> Finis

因为在代码里面，我们把 Agent 的 Verbose 模式打开了，所以在输出结果里面，你可以直接看到 Agent 整个思考的日志。从这里，你会发现几个有意思的现象。 

第一个，是 Agent 每一步的操作，可以分成 5 个步骤，分别是 Action、Action Input、Observation、Thought，最后输出一个 Final Answer。 

Action，就是根据用户的输入，选择应该选取哪一个 Tool，然后行动。 

Action Input，就是根据需要使用的 Tool，从用户的输入里提取出相关的内容，可以输入到 Tool 里面。 

Oberservation，就是观察通过使用 Tool 得到的一个输出结果。 

Thought，就是再看一眼用户的输入，判断一下该怎么做。 

Final Answer，就是 Thought 在看到 Obersavation 之后，给出的最终输出。

这一整个过程，其实也是通过一段 Prompt 来实现的，你可以去看一下 Langchain 源码里
# flake8: noqa
PREFIX = """Answer the following questions as best you can. You have access to the following tools:"""
FORMAT_INSTRUCTIONS = """Use the following format:
  
Question: the input question you must answer

Thought: you should always think about what to do

Action: the action to take, should be one of [{tool_names}]

Action Input: the input to the action

Observation: the result of the action

... (this Thought/Action/Action Input/Observation can repeat N times)

Thought: I now know the final answer

Final Answer: the final answer to the original input question"""

SUFFIX = """Begin!
  
Question: {input}

Thought:{agent_scratchpad}"""


In [14]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.text_splitter import SpacyTextSplitter
from langchain import OpenAI, VectorDBQA
from langchain.document_loaders import TextLoader

llm = OpenAI(temperature=0)
loader = TextLoader('./data/demo.txt')
documents = loader.load()
text_splitter = SpacyTextSplitter(chunk_size=256, pipeline="zh_core_web_sm")
texts = text_splitter.split_documents(documents)

embeddings = OpenAIEmbeddings()
docsearch = FAISS.from_documents(texts, embeddings)

faq_chain = VectorDBQA.from_chain_type(llm=llm, vectorstore=docsearch, verbose=True)


/var/folders/7l/brdxy3kj6c9g9xvg5k_0kw3h0000gn/T/ipykernel_60156/2211358665.py:13: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings = OpenAIEmbeddings()
/Users/zouguoyang/opt/anaconda3/envs/langchain/lib/python3.9/site-packages/langchain/chains/retrieval_qa/base.py:316: UserWarning: `VectorDBQA` is deprecated - please use `from langchain.chains import RetrievalQA`
  warnings.warn(


In [23]:
print(faq_chain.run("萧薰儿是谁，你用的什么gpt模型"))



> Entering new VectorDBQA chain...

> Finished chain.
 萧薰儿是一位身着紫色衣裙的少女，她是一个有着清冷淡然气质的人，被认为是家族中年轻一辈的第一人。我使用的是GPT-3模型。


In [18]:
from langchain.agents import tool

@tool("FAQ")
def faq(intput: str) -> str:
    """"useful for information extract for a chinese novel."""
    return faq_chain.run(intput)

tools = [
    Tool(
        name = "Search Order",func=search_order, 
        description="useful for when you need to answer questions about customers orders"
    ),
    Tool(name="Recommend Product", func=recommend_product, 
         description="useful for when you need to answer questions about product recommendations"
    ),
    faq
]

agent = initialize_agent(tools, llm, agent="zero-shot-react-description", verbose=True)


In [19]:
question = "萧炎是谁？"
result = agent.run(question)
print(result)




> Entering new AgentExecutor chain...
 I should use the FAQ tool to extract information about a Chinese novel.
Action: FAQ
Action Input: 萧炎

> Entering new VectorDBQA chain...

> Finished chain.

Observation:  萧炎是三年前意气风发的少年，现在已经不在同一个阶层，但仍然被萧媚所关注。他曾经在自己面前露出过最谦卑的笑容，如今想要讨还回去。
Thought: I now know the final answer.
Final Answer: 萧炎是三年前意气风发的少年，现在已经不在同一个阶层，但仍然被萧媚所关注。他曾经在自己面前露出过最谦卑的笑容，如今想要讨还回去。

> Finished chain.
萧炎是三年前意气风发的少年，现在已经不在同一个阶层，但仍然被萧媚所关注。他曾经在自己面前露出过最谦卑的笑容，如今想要讨还回去。


下面这个例子可以保存聊天记录，并且回答，支持多轮对话，下面例子跑不通，可以去看第17讲

In [ ]:
import re

answer_order_info = PromptTemplate(
    template="请把下面的订单信息回复给用户： \n\n {order}?", input_variables=["order"]
)
answer_order_llm = LLMChain(llm=ChatOpenAI(temperature=0),  prompt=answer_order_info)

@tool("Search Order", return_direct=True)
def search_order(input:str)->str:
    """useful for when you need to answer questions about customers orders"""
    pattern = r"\d+[A-Z]+"
    match = re.search(pattern, input)

    order_number = input
    if match:
        order_number = match.group(0)
    else:
        return "请问您的订单号是多少？"
    if order_number == ORDER_1:        
        return answer_order_llm.run(json.dumps(ORDER_1_DETAIL))
    elif order_number == ORDER_2:
        return answer_order_llm.run(json.dumps(ORDER_2_DETAIL))
    else:
        return f"对不起，根据{input}没有找到您的订单"

from langchain.memory import ConversationBufferMemory
from langchain.chat_models import ChatOpenAI

tools = [search_order,recommend_product, faq]
chatllm=ChatOpenAI(temperature=0)
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
conversation_agent = initialize_agent(tools, chatllm, 
                                      agent="conversational-react-description", 
                                      memory=memory, verbose=True)
